In [1]:
# !pip install -q espnet
# !pip install -q espnet_model_zoo

In [2]:
# !pip install gdown

In [3]:
# !ls

In [4]:
# from pydub import AudioSegment

# def split_audio_into_equal_parts(input_file, output_part1, output_part2):
#     audio = AudioSegment.from_wav(input_file)

#     # Calculate the midpoint of the audio
#     midpoint = len(audio) // 2

#     # Split the audio into two equal parts
#     part1 = audio[:midpoint]
#     part2 = audio[midpoint:]

#     # Export the two parts
#     part1.export(output_part1, format="wav")
#     part2.export(output_part2, format="wav")

# # Example usage
# split_audio_into_equal_parts("output_part1.wav", "output_part3.wav", "output_part4.wav")


In [55]:
# Download one utterance from real noisy speech of CHiME4
# !gdown --id 1SmrN5NFSg6JuQSs2sfy3ehD8OIcqK6wS -O ./speech_sc_enhance/M05_440C0213_PED_REAL.wav
import os
import torch
import soundfile
from IPython.display import display, Audio
%cd /home/suryansh/MADHAV
mixwav_mc, sr = soundfile.read("./hindi_data/givenPMS/newGivenPMS/splitAudioFinal/PM_2022_april_01.wav")
# mixwav.shape: num_samples, num_channels
mixwav_sc = mixwav_mc[:,1]
# display(Audio(mixwav_mc.T, rate=sr))
torch.cuda.empty_cache()

/home/suryansh/MADHAV


In [56]:
mixwav_mc2=mixwav_mc[:100*sr, :]

In [57]:
# !gdown --id 17DMWdw84wF3fz3t7ia1zssdzhkpVQGZm -O ./speech_sc_enhance/chime_tasnet_singlechannel.zip
# !unzip ./speech_sc_enhance/chime_tasnet_singlechannel.zip -d ./speech_sc_enhance/enh_model_sc

In [58]:
# Load the model
# If you encounter error "No module named 'espnet2'", please re-run the 1st Cell. This might be a colab bug.
import sys
import soundfile
from espnet2.bin.enh_inference import SeparateSpeech


separate_speech = {}
# For models downloaded from GoogleDrive, you can use the following script:
enh_model_sc = SeparateSpeech(
  train_config="./speech_sc_enhance/enh_model_sc/exp/enh_train_enh_conv_tasnet_raw/config.yaml",
  model_file="./speech_sc_enhance/enh_model_sc/exp/enh_train_enh_conv_tasnet_raw/5epoch.pth",
  # for segment-wise process on long speech
  normalize_segment_scale=False,
  show_progressbar=True,
  ref_channel=1,
  normalize_output_wav=True,
  device="cuda:0",
)

In [59]:
# play the enhanced single-channel speech
wave = enh_model_sc(mixwav_sc[None, ...], sr)

print("Input real noisy speech", flush=True)
display(Audio(mixwav_sc, rate=sr))
print("Enhanced speech", flush=True)
display(Audio(wave[0].squeeze(), rate=sr))

Input real noisy speech


Enhanced speech


## Multichannel audio enhancement

In [15]:
# Download the pretained enhancement model

!gdown --id 1FohDfBlOa7ipc9v2luY-QIFQ_GJ1iW_i -O ./speech_sc_enhance/mvdr_beamformer_16k_se_raw_valid.zip
!unzip ./speech_sc_enhance/mvdr_beamformer_16k_se_raw_valid.zip -d ./speech_sc_enhance/enh_model_mc

/home/suryansh/.local/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "/usr/lib/python3/dist-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/usr/lib/python3/dist-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
OSError: [Errno 113] No route to host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 700, in urlopen
    httplib_response = self._make_request(
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 383, in _make_request
    self._va

In [16]:
# Load the model
# If you encounter error "No module named 'espnet2'", please re-run the 1st Cell. This might be a colab bug.
import sys
import soundfile
from espnet2.bin.enh_inference import SeparateSpeech


separate_speech = {}
# For models downloaded from GoogleDrive, you can use the following script:
enh_model_mc = SeparateSpeech(
  train_config="./speech_sc_enhance/enh_model_mc/exp/enh_train_enh_beamformer_mvdr_raw/config.yaml",
  model_file="./speech_sc_enhance/enh_model_mc/exp/enh_train_enh_beamformer_mvdr_raw/11epoch.pth",
  # for segment-wise process on long speech
  normalize_segment_scale=False,
  show_progressbar=True,
  ref_channel=4,
  normalize_output_wav=True,
  device="cuda:0",
)


In [17]:
wave = enh_model_mc(mixwav_mc2[None, ...], sr)
print("Input real noisy speech", flush=True)
display(Audio(mixwav_mc.T, rate=sr))
print("Enhanced speech", flush=True)
display(Audio(wave[0].squeeze(), rate=sr))

IndexError: index 4 is out of bounds for dimension 1 with size 2

## Testing with and without audio enhancement

In [26]:
%cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500

/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500


In [27]:
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


d = ModelDownloader()
# It may takes a while to download and build models
speech2text = Speech2Text(
    "exp/asr_train_asr_raw_hindi_bpe500/config.yaml",
    "exp/asr_train_asr_raw_hindi_bpe500/valid.acc.ave_10best.pth",
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.5,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

In [48]:
%cd /home/suryansh/MADHAV
speech, sr=soundfile.read("./audio_enhancement_testing/Hindi-dopahar-samachar-20238414335_2.wav")
%cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500
nbests = speech2text(speech)
text, *_ = nbests[0]
preds=(text_normalizer(text))
preds

/home/suryansh/MADHAV
/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500


'और आईआईएम संशोधन मीडिया पर उच्चतम न्यायालय ने आपराधिक मझहरी मंडी में कौगशीता राहुल गांधी की सदा पर रोक लगाई और सांसद का डर जाबा'

In [49]:
%cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500
nbests = speech2text(wave[0].squeeze())
text, *_ = nbests[0]
preds=(text_normalizer(text))
preds

/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500


'और आईआईएम संशोधन विधेयक पारित उच्चतम न्यायालय ने आपराधिक मानहारी मामले में कांग्रेस नेता राहुल गांधी की खदा पर रोक लगाई और संसद का दर्जा बया'